# Self-Driving Car Engineer Nanodegree

## Deep Learning

## Project: Build a Traffic Sign Recognition Classifier

In this notebook, a template is provided for you to implement your functionality in stages which is required to successfully complete this project. If additional code is required that cannot be included in the notebook, be sure that the Python code is successfully imported and included in your submission, if necessary. Sections that begin with **'Implementation'** in the header indicate where you should begin your implementation for your project. Note that some sections of implementation are optional, and will be marked with **'Optional'** in the header.

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

---
## Step 0: Load The Data

In [ ]:
# Load pickled data
import pickle
import os

# need to have this to get data
DATA_LOCATION = 'data/traffic-signs/'

# can create this on-the-fly, ok if already created
TENSOR_PATH = DATA_LOCATION + 'tf/'
os.makedirs(TENSOR_PATH, exist_ok = True)

training_file = 'train.p'
testing_file = 'test.p'

with open(DATA_LOCATION + training_file, mode='rb') as f:
    train = pickle.load(f)
train_features, train_labels = train['features'], train['labels']
    
with open(DATA_LOCATION + testing_file, mode='rb') as f:
    test = pickle.load(f)
X_test, y_test = test['features'], test['labels']


---

## Step 1: Dataset Summary & Exploration

The pickled data is a dictionary with 4 key/value pairs:

- `'features'` is a 4D array containing raw pixel data of the traffic sign images, (num examples, width, height, channels).
- `'labels'` is a 2D array containing the label/class id of the traffic sign. The file `signnames.csv` contains id -> name mappings for each id.
- `'sizes'` is a list containing tuples, (width, height) representing the the original width and height the image.
- `'coords'` is a list containing tuples, (x1, y1, x2, y2) representing coordinates of a bounding box around the sign in the image. **THESE COORDINATES ASSUME THE ORIGINAL IMAGE. THE PICKLED DATA CONTAINS RESIZED VERSIONS (32 by 32) OF THESE IMAGES**

Complete the basic data summary below.

In [ ]:
### Replace each question mark with the appropriate value.

# TODO: Number of training examples
n_train = len(train_features)

# TODO: Number of testing examples.
n_test = len(X_test)

# TODO: What's the shape of an traffic sign image?
image_shape = (len(train_features[0]), len(train_features[0][0]), len(train_features[0][0][0]))

labels = {}
for label in train_labels:
    if (not labels.get(label)):
        labels[label] = 0
    labels[label] += 1
    
# TODO: How many unique classes/labels there are in the dataset.
n_classes = len(labels)

print("Number of training examples =", n_train)
print("Number of testing examples =", n_test)
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)


Visualize the German Traffic Signs Dataset using the pickled file(s). This is open ended, suggestions include: plotting traffic sign images, plotting the count of each sign, etc.

The [Matplotlib](http://matplotlib.org/) [examples](http://matplotlib.org/examples/index.html) and [gallery](http://matplotlib.org/gallery.html) pages are a great resource for doing visualizations in Python.

**NOTE:** It's recommended you start with something simple first. If you wish to do more, come back to it after you've completed the rest of the sections.

In [ ]:
### Data exploration visualization goes here.
### Feel free to use as many code cells as needed.
import random
import numpy as np
import cv2
import matplotlib.pyplot as plt

# Visualizations will be shown in the notebook.
%matplotlib inline

In [ ]:

def max_contrast(img):
    lab_img = cv2.cvtColor(img, cv2.COLOR_RGB2LAB)
    lab_img[:,:,0] = cv2.equalizeHist(lab_img[:,:,0])
    return cv2.cvtColor(lab_img, cv2.COLOR_LAB2RGB)

def image_pair(img_in):
    contrast = max_contrast(img_in)
    return np.hstack((img_in.squeeze(), contrast.squeeze()))

print("Random image:")

for i in range(8):
    index = random.randint(0, len(train_features))
    image = train_features[index]

    plt.figure(figsize=(4,4))
    plt.imshow(image_pair(image))

#plt.show()

In [ ]:
print("Label frequency:")
plt.plot(list(labels.keys()),list(labels.values()))
plt.show()

In [ ]:
import operator
print("Descending by frequency:")
labels_by_frequency = list(map(lambda x: x[1], sorted(labels.items(), key=operator.itemgetter(1), reverse = True)))
plt.plot(labels_by_frequency)
plt.show()

In [ ]:
print("Data:")
freq_min, freq_max, _, _ = cv2.minMaxLoc(np.array(list(labels.values())))
print("Min frequency:", freq_min, "\nMax frequency:", freq_max, "\nRatio:", freq_max / freq_min, "\n")
for k in labels.keys():
    print("Label: {k}, Frequency: {v}".format_map({'k': k, 'v': labels[k]}))


----

## Step 2: Design and Test a Model Architecture

Design and implement a deep learning model that learns to recognize traffic signs. Train and test your model on the [German Traffic Sign Dataset](http://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset).

There are various aspects to consider when thinking about this problem:

- Neural network architecture
- Play around preprocessing techniques (normalization, rgb to grayscale, etc)
- Number of examples per label (some have more than others).
- Generate fake data.

Here is an example of a [published baseline model on this problem](http://yann.lecun.com/exdb/publis/pdf/sermanet-ijcnn-11.pdf). It's not required to be familiar with the approach used in the paper but, it's good practice to try to read papers like these.

**NOTE:** The LeNet-5 implementation shown in the [classroom](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) at the end of the CNN lesson is a solid starting point. You'll have to change the number of classes and possibly the preprocessing, but aside from that it's plug and play!

### Implementation

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project. Once you have completed your implementation and are satisfied with the results, be sure to thoroughly answer the questions that follow.

In [ ]:
### Preprocess the data here.
### Feel free to use as many code cells as needed.

import math

_image_center = (image_shape[0] / 2, image_shape[1] / 2)

_rotation_list = []
for r in range(10,2,-2):
    _rotation_list.append(cv2.getRotationMatrix2D(_image_center, r, 1))
    _rotation_list.append(cv2.getRotationMatrix2D(_image_center, -r, 1))

_translation_list = []
# shift up to 10% of image size
xrange = math.ceil(image_shape[0] / 10)
yrange = math.ceil(image_shape[1] / 10)
for y in range(-yrange, yrange, 2):
    for x in range(-xrange, xrange, 2):
        M = np.float32([
            [1, 0, x],
            [0, 1, y]
        ])
        _translation_list.append(M)

def do_warps(images, transform_list):
    warps = []
    for img in images:
        for transform in transform_list:
            warps.append(cv2.warpAffine(img, transform, img.shape[:2]))
    return warps

def do_translations(images):
    return do_warps(images, _translation_list)

def do_rotations(images):
    return do_warps(images, _rotation_list)

def do_noise(images, out_count = 8):
    noisy = []
    for img in images:
        w,h,d = img.shape
        for i in range(out_count):
            noise = np.random.normal(0,0.05,(w,h,d))
            noised = img + img * noise
            noisy.append(noised)
    return noisy
#
### Generate multiple output images based on a single input
### Perform various rotations and projections from input image
### Input is a single image, output is original + N transformed images
### Transforms are multiplicative - N = rotations * translations * noises
def random_warp(img):
    warped = [img]
    
    warped = do_rotations(warped)
    warped = do_translations(warped)
    # noise function is broken in some weird way
    # it ends up blowing up AND changing the colorspace of the original?!
    #warped = do_noise(warped, 2)
    return warped
    
def compare(img_in):
    alts = random_warp(img_in)
    
    return np.hstack((img_in.squeeze(), alts[0].squeeze(), alts[1].squeeze()))

print("Random image:")

for i in range(3):
    index = random.randint(0, len(train_features) - 1)
    image = max_contrast(train_features[index])

    plt.figure(figsize=(4,4))
    plt.imshow(compare(image))


### Question 1 

_Describe how you preprocessed the data. Why did you choose that technique?_

**Answer:**



Input images are split into test and training immediately, with approximately 10% of each category being set aside for validation. _(**TODO** should this be a fixed number per category?)_ Remaining images in the training set are processed to increase their quantity 32x via rotation, translation and noise addition (noise not working 2017-01-20).  This is performed by `warpTransform` with values for rotation +/- 5 and translation +/- 10% of image size.  This is substantially similar to [Howard](https://ai2-s2-pdfs.s3.amazonaws.com/d671/75d17c450ab0ac9c256103828f9e9a0acb85.pdf) for the initial image processing step before subsequent scaling and cropping.

On input to training or testing, data is processed to maximize contrast. This is performed by transforming to LAB colorspace, running a histogram equalization over the L channel and transforming back to RGB. [Cireșan, et al](http://people.idsia.ch/~juergen/nn2012traffic.pdf) suggests a similar approach, but with additional local histogram equalization and contrast enhancement. 

In [ ]:
### Generate data additional data (OPTIONAL!)
### and split the data into training/validation/testing sets here.
### Feel free to use as many code cells as needed.

### Question 2

_Describe how you set up the training, validation and testing data for your model. **Optional**: If you generated additional data, how did you generate the data? Why did you generate the data? What are the differences in the new dataset (with generated data) from the original dataset?_

**Answer:**

In [ ]:
import tensorflow as tf

EPOCHS = 50
BATCH_SIZE = 64

from tensorflow.contrib.layers import flatten

def activate(x):
    return tf.nn.relu(x)

def pool(layer, pool_size, stride=-1):
    if (stride < 0):
        stride = pool_size
    # TODO: depth for 1?
    return tf.nn.max_pool(layer, ksize=[1, pool_size, pool_size, 1], strides=[1, stride, stride, 1], padding='VALID')
    

def fc_layer(prev_layer, o_size, mu=0, sigma=0.1):
    i_size = prev_layer.get_shape()[1].value
    layer_weights = tf.Variable(tf.truncated_normal(shape=(i_size, o_size), mean = mu, stddev = sigma))
    layer_biases = tf.Variable(tf.zeros(o_size))
    return tf.matmul(prev_layer, layer_weights) + layer_biases
    
def conv_layer(prev_layer, o_depth, k_size=5, stride=1, mu=0, sigma=0.1):
    i_depth = prev_layer.get_shape()[3].value
    layer_weights = tf.Variable(tf.truncated_normal(shape=(k_size, k_size, i_depth, o_depth), mean = mu, stddev = sigma))
    layer_biases = tf.Variable(tf.zeros(o_depth))
    return tf.nn.conv2d(prev_layer, layer_weights, strides=[1, stride, stride, 1], padding='VALID') + layer_biases
    
def LeNet(x, keep_prob):    
    # Hyperparameters
    mu = 0
    sigma = 0.1

    # Layer 1: Convolutional. Input = 32x32x3. Output = 28x28x6.
    # Pooling. Input = 28x28x6. Output = 14x14x6.
    layer1 = activate(conv_layer(x, 6))
    layer1 = pool(layer1, 2)

    # Layer 2: Convolutional. Output = 10x10x16.
    # Pooling. Input = 10x10x16. Output = 5x5x16.
    layer2 = activate(conv_layer(layer1, 16))
    layer2 = pool(layer2, 2)

    # Flatten. Input = 5x5x16. Output = 400.
    layer2 = flatten(layer2)    

    # Layer 3: Fully Connected. Input = 400. Output = 120.
    layer3 = activate(fc_layer(layer2, 120, mu, sigma))

    # Layer 4: Fully Connected. Input = 120. Output = 84.
    layer4 = activate(fc_layer(layer3, 84, mu, sigma))

    # Dropout to avoid overfitting
    layer4 = tf.nn.dropout(layer4, keep_prob)
    
    # Layer 5: Fully Connected. Input = 84. Output = 10.
    # NO ACTIVATION
    logits = fc_layer(layer4, n_classes, mu, sigma)

    return logits

In [ ]:
# features
x = tf.placeholder(tf.float32, (None, image_shape[0], image_shape[1], image_shape[2]))
# labels
y = tf.placeholder(tf.int32, (None))
# dropout
k = tf.placeholder(tf.float32, (None))

one_hot_y = tf.one_hot(y, n_classes)

LEARNING_RATE = 0.001

logits = LeNet(x, k)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits, one_hot_y)
loss_operation = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate = LEARNING_RATE)
training_operation = optimizer.minimize(loss_operation)

correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
saver = tf.train.Saver(max_to_keep = 100, name = 'traffic_signs', restore_sequentially = True)

def evaluate(X_data, y_data):
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
        accuracy = sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y, k: 1.0})
        total_accuracy += (accuracy * len(batch_x))
    return total_accuracy / num_examples

### Question 3

_What does your final architecture look like? (Type of model, layers, sizes, connectivity, etc.)  For reference on how to build a deep neural network using TensorFlow, see [Deep Neural Network in TensorFlow
](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/b516a270-8600-4f93-a0a3-20dfeabe5da6/concepts/83a3a2a2-a9bd-4b7b-95b0-eb924ab14432) from the classroom._


**Answer:**

In [ ]:
from sklearn.utils import shuffle

VALIDATION_SET_PERCENT = 10
validation_set_size = math.floor(len(train_features) * (VALIDATION_SET_PERCENT / 100))

best_accuracy = 0.0
DROPOUT_KEEP_PROB = 0.5

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    print("Training...")
    print()
    for i in range(1, EPOCHS + 1):
        train_features, train_labels = shuffle(train_features, train_labels)

        X_validation, y_validation = train_features[:validation_set_size], train_labels[:validation_set_size]
        X_train, y_train = train_features[validation_set_size:], train_labels[validation_set_size:]
        
        # DO DATA EXPANSION HERE
        
        num_examples = len(X_train)

        for offset in range(0, num_examples, BATCH_SIZE):
            end = offset + BATCH_SIZE
            batch_x, batch_y = X_train[offset:end], y_train[offset:end]
            sess.run(training_operation, feed_dict={x: batch_x, y: batch_y, k: DROPOUT_KEEP_PROB})

        saver.save(sess, TENSOR_PATH + 'traffic_sign_TEST')
        validation_accuracy = evaluate(X_validation, y_validation)
        test_accuracy = evaluate(X_test, y_test)
        saver.restore(sess, TENSOR_PATH + 'traffic_sign_TEST')
        
        print("EPOCH {}, Validation Accuracy = {:.1f}, Test Accuracy = {:.1f}".format(i, validation_accuracy * 100, test_accuracy * 100))

        if validation_accuracy > best_accuracy:
            best_accuracy = validation_accuracy
            saver.save(sess, TENSOR_PATH + 'best', global_step = i)
            print("Improved model saved")

        print()

    # keep this in session
    saver.save(sess, TENSOR_PATH + 'final')
    print("Final model saved")

print("Final accuracy = {:.1f}, best = {:.1f}".format(validation_accuracy * 100, best_accuracy * 100))


### Question 4

_How did you train your model? (Type of optimizer, batch size, epochs, hyperparameters, etc.)_


**Answer:**

reference http://sebastianruder.com/optimizing-gradient-descent/index.html#visualizationofalgorithms for selecting optimizer (use the one we had before, but confirmed it's probably the best). Batch size

### Question 5


_What approach did you take in coming up with a solution to this problem? It may have been a process of trial and error, in which case, outline the steps you took to get to the final solution and why you chose those steps. Perhaps your solution involved an already well known implementation or architecture. In this case, discuss why you think this is suitable for the current problem._

**Answer:**

---

## Step 3: Test a Model on New Images

Take several pictures of traffic signs that you find on the web or around you (at least five), and run them through your classifier on your computer to produce example results. The classifier might not recognize some local signs but it could prove interesting nonetheless.

You may find `signnames.csv` useful as it contains mappings from the class id (integer) to the actual sign name.

### Implementation

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project. Once you have completed your implementation and are satisfied with the results, be sure to thoroughly answer the questions that follow.

In [ ]:
### Load the images and plot them here.
### Feel free to use as many code cells as needed.

### Question 6

_Choose five candidate images of traffic signs and provide them in the report. Are there any particular qualities of the image(s) that might make classification difficult? It could be helpful to plot the images in the notebook._



**Answer:**

In [ ]:
### Run the predictions here.
### Feel free to use as many code cells as needed.

### Question 7

_Is your model able to perform equally well on captured pictures when compared to testing on the dataset? The simplest way to do this check the accuracy of the predictions. For example, if the model predicted 1 out of 5 signs correctly, it's 20% accurate._

_**NOTE:** You could check the accuracy manually by using `signnames.csv` (same directory). This file has a mapping from the class id (0-42) to the corresponding sign name. So, you could take the class id the model outputs, lookup the name in `signnames.csv` and see if it matches the sign from the image._


**Answer:**

In [ ]:
### Visualize the softmax probabilities here.
### Feel free to use as many code cells as needed.

### Question 8

*Use the model's softmax probabilities to visualize the **certainty** of its predictions, [`tf.nn.top_k`](https://www.tensorflow.org/versions/r0.12/api_docs/python/nn.html#top_k) could prove helpful here. Which predictions is the model certain of? Uncertain? If the model was incorrect in its initial prediction, does the correct prediction appear in the top k? (k should be 5 at most)*

`tf.nn.top_k` will return the values and indices (class ids) of the top k predictions. So if k=3, for each sign, it'll return the 3 largest probabilities (out of a possible 43) and the correspoding class ids.

Take this numpy array as an example:

```
# (5, 6) array
a = np.array([[ 0.24879643,  0.07032244,  0.12641572,  0.34763842,  0.07893497,
         0.12789202],
       [ 0.28086119,  0.27569815,  0.08594638,  0.0178669 ,  0.18063401,
         0.15899337],
       [ 0.26076848,  0.23664738,  0.08020603,  0.07001922,  0.1134371 ,
         0.23892179],
       [ 0.11943333,  0.29198961,  0.02605103,  0.26234032,  0.1351348 ,
         0.16505091],
       [ 0.09561176,  0.34396535,  0.0643941 ,  0.16240774,  0.24206137,
         0.09155967]])
```

Running it through `sess.run(tf.nn.top_k(tf.constant(a), k=3))` produces:

```
TopKV2(values=array([[ 0.34763842,  0.24879643,  0.12789202],
       [ 0.28086119,  0.27569815,  0.18063401],
       [ 0.26076848,  0.23892179,  0.23664738],
       [ 0.29198961,  0.26234032,  0.16505091],
       [ 0.34396535,  0.24206137,  0.16240774]]), indices=array([[3, 0, 5],
       [0, 1, 4],
       [0, 5, 1],
       [1, 3, 5],
       [1, 4, 3]], dtype=int32))
```

Looking just at the first row we get `[ 0.34763842,  0.24879643,  0.12789202]`, you can confirm these are the 3 largest probabilities in `a`. You'll also notice `[3, 0, 5]` are the corresponding indices.

**Answer:**

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.